# Feature extraction for time series 

In [3]:
import sys
import os

module_path = os.path.abspath(os.path.join('..'))
sys.path.insert(1, module_path)
import utility
import csv  

import pandas as pd
import time
import random
from PyEMD import EEMD
from PyEMD import EMD
import librosa
import numpy as np
import pywt
from scipy.stats import skew 

In [6]:
N_IMFS = 10
N_SIFTS = 15
SELECT_IMFS = [4,5,6,7]
WAVELET = 'db8'
LEVELS_DWT = 10
N_SAMPLES = 2
TARGET_RATE = 40000
DECOMP = 'EEMD'

def get_n_best_IMFs(timeseries, sr, method = 'EMD'):
    m_trials = 2
    result = []
    t = utility.get_t(timeseries, sr)
    if method == 'EEMD':
        eemd = EEMD(trials=m_trials)
        eemd.spline_kind="slinear"
        eemd.FIXE = N_SIFTS
        # Execute EEMD on S
        IMFs = eemd.eemd(timeseries, t, max_imf = N_IMFS)
    else:
        emd = EMD()
        emd.spline_kind="slinear"
        emd.FIXE = N_SIFTS
        # Execute EEMD on S
        IMFs = emd.emd(timeseries, t, max_imf = N_IMFS)
    for idx in range(len(IMFs)):
        if idx in SELECT_IMFS:
            result.append(IMFs[idx])

    return result

def get_n_best_levels(timeseries):
    dwtValues = pywt.wavedec(timeseries, WAVELET, level=LEVELS_DWT)
    len_D = len(dwtValues)
    result = []
    for n, c_D in enumerate(dwtValues):
        if n == 0:
            continue
        result.append(c_D)
    return result

def get_entropy(timeseries):
    timeseries_nz = timeseries[timeseries != 0]
    return - np.sum(((timeseries_nz**2)*np.log(timeseries_nz**2)))
    
def get_energy(timeseries):  
    N = len(timeseries)
    return np.sum(np.abs(timeseries) ** 2) / N
    

def get_features(timeseries, sr):
    ft2 = librosa.feature.zero_crossing_rate(timeseries)[0]
    ft3 = librosa.feature.spectral_rolloff(timeseries)[0]
    ft4 = librosa.feature.spectral_centroid(timeseries)[0]
    ft5 = librosa.feature.spectral_contrast(timeseries)[0]
    ft6 = librosa.feature.spectral_bandwidth(timeseries)[0]

    ### Get HOS and simple features 
    ft0_trunc = np.hstack((np.mean(timeseries), np.std(timeseries),
                           skew(timeseries), np.max(timeseries),
                           np.median(timeseries), np.min(timeseries),
                           get_energy(timeseries), get_entropy(timeseries)))
  
    ### Spectral Features 
    ft2_trunc = np.hstack((np.mean(ft2), np.std(ft2), skew(ft2), np.max(ft2), np.median(ft2), np.min(ft2)))
    ft3_trunc = np.hstack((np.mean(ft3), np.std(ft3), skew(ft3), np.max(ft3), np.median(ft3), np.min(ft3)))
    ft4_trunc = np.hstack((np.mean(ft4), np.std(ft4), skew(ft4), np.max(ft4), np.median(ft4), np.min(ft4)))
    ft5_trunc = np.hstack((np.mean(ft5), np.std(ft5), skew(ft5), np.max(ft5), np.median(ft5), np.min(ft5)))
    ft6_trunc = np.hstack((np.mean(ft6), np.std(ft6), skew(ft6), np.max(ft6), np.median(ft6), np.max(ft6)))
    return np.hstack((ft0_trunc, ft2_trunc, ft3_trunc, ft4_trunc, ft5_trunc, ft6_trunc))


def get_row(timeseries, sr):
    if DECOMP == 'noDecomp':
        return get_features(timeseries,sr)
    elif DECOMP in ['EMD','EEMD']:
        IMFS = get_n_best_IMFs(timeseries, sr, method = DECOMP)
        all_features = get_features(IMFS[0],sr)
        for n in range(1,len(IMFS)):
            print(all_features)
            all_features = np.concatenate([all_features, get_features(IMFS[n],sr)])
        return all_features
    elif DECOMP == 'DWT':
        levels = get_n_best_levels(timeseries)
        all_features = get_features(levels[0],sr)
        for n in range(1,len(levels)):
            print(all_features)
            all_features = np.concatenate([all_features, get_features(levels[n],sr)])
        return all_features
    elif DECOMP in ['EMD_DWT', 'EEMD_DWT']:
        method = 'EMD'
        if DECOMP == 'EEMD_DWT': method = 'EEMD'
        imf_levels = []
        
        IMFS = get_n_best_IMFs(timeseries, sr, method = method)
        for imf in IMFS:
            levels = get_n_best_levels(imf)
            for level in levels:
                imf_levels.append(level)
        
        all_features = get_features(imf_levels[0],sr)
        for n in range(1,len(imf_levels)):
            print(all_features)
            all_features = np.concatenate([all_features, get_features(imf_levels[n],sr)])
                
        return all_features
    return False


def add_row_csv(timeseries, sr, label, filename, csv_filename):
        row = get_row(timeseries, sr)
        row = np.concatenate([row.astype(str), np.array([filename, label])])
        if os.path.exists(csv_filename):
            with open(csv_filename, 'a') as f:
                writer = csv.writer(f)
                writer.writerow(row)
        else:
            with open(csv_filename, 'w') as f:
                writer = csv.writer(f)
                writer.writerow(row)
                
def read_audio_write_features(path, label, filename, csv_filename):
        sr, y = utility.read_wav_file(path, TARGET_RATE)
        y = utility.denoise_audio(y)
        y, sr = utility.downsample(y, sr)
        add_row_csv(y, sr, label, filename, csv_filename)

def get_dataset(filepath, labels, csv_filename):
    '''
    Creating a dataset, specifically for audio-files
    '''
    random.seed(20)
    filepath1 = filepath + '/' + labels[0]
    filepath2 = filepath + '/' + labels[1]
    # Assumes that the data beloning to a label is saved in seperate directories 
    label1 = os.listdir(filepath1) 
    label2 = os.listdir(filepath2)
    
    label1_idx = random.sample(range(len(label1)), N_SAMPLES)
    label2_idx = random.sample(range(len(label1)), N_SAMPLES)
    
    for i in range(0,N_SAMPLES):
        print(f'Processing : sample number {i} of {N_SAMPLES}')
        filename1 , filename2 =  label1[label1_idx[i]], label2[label2_idx[i]]
        read_audio_write_features(filepath1 + '/' + filename1, labels[0], filename1, csv_filename)
        read_audio_write_features(filepath2 + '/' +  filename2, labels[1],filename2, csv_filename)

In [7]:
start = time.time()
filepath = '../../data'
labels = ['crackle','no-crackle']
get_dataset(filepath, labels, DECOMP+'.csv')
print(f' Processing finished, total time used = {time.time() - start}')


Processing : sample number 0 of 2
[-9.12791069e-04  1.36613984e-01 -9.43718583e-02  7.06168256e-01
 -1.05346670e-04 -7.12894528e-01  1.86642137e-02  1.11979492e+03
  5.33573545e-02  1.26647721e-02 -9.12237768e-01  6.98242188e-02
  5.76171875e-02  2.49023438e-02  8.13064049e+02  2.26938964e+02
 -1.28733655e-01  1.22739258e+03  8.23645020e+02  4.30664062e+02
  5.34693844e+02  1.49477813e+02  1.68307772e-01  8.08002392e+02
  5.16136786e+02  3.06636625e+02  1.04707202e+01  4.87541278e+00
  5.14292235e-01  2.43810354e+01  1.00066824e+01  7.84052085e-01
  3.85652981e+02  8.17242228e+01 -2.14294162e-01  5.28094059e+02
  3.96931494e+02  5.28094059e+02]
[-9.12791069e-04  1.36613984e-01 -9.43718583e-02  7.06168256e-01
 -1.05346670e-04 -7.12894528e-01  1.86642137e-02  1.11979492e+03
  5.33573545e-02  1.26647721e-02 -9.12237768e-01  6.98242188e-02
  5.76171875e-02  2.49023438e-02  8.13064049e+02  2.26938964e+02
 -1.28733655e-01  1.22739258e+03  8.23645020e+02  4.30664062e+02
  5.34693844e+02  1.49

[-2.94577607e-04  7.87204641e-02  2.47586924e-02  3.79011104e-01
 -5.80855484e-04 -3.73110138e-01  6.19699824e-03  4.01949284e+02
  4.34709821e-02  5.69486998e-03 -8.26814734e-01  5.22460938e-02
  4.44335938e-02  2.63671875e-02  7.48740234e+02  1.36123299e+02
 -1.42702438e-01  1.01206055e+03  7.42895508e+02  4.95263672e+02
  4.99506561e+02  6.92074963e+01  9.39424124e-01  7.29732591e+02
  4.92372450e+02  3.90361968e+02  1.01861490e+01  2.80859405e+00
  4.73575222e-03  1.69122990e+01  9.93375365e+00  3.37158449e+00
  3.60405546e+02  6.48053666e+01  1.36414160e-01  5.21405423e+02
  3.71925797e+02  5.21405423e+02]
[-2.94577607e-04  7.87204641e-02  2.47586924e-02  3.79011104e-01
 -5.80855484e-04 -3.73110138e-01  6.19699824e-03  4.01949284e+02
  4.34709821e-02  5.69486998e-03 -8.26814734e-01  5.22460938e-02
  4.44335938e-02  2.63671875e-02  7.48740234e+02  1.36123299e+02
 -1.42702438e-01  1.01206055e+03  7.42895508e+02  4.95263672e+02
  4.99506561e+02  6.92074963e+01  9.39424124e-01  7.2973